In [176]:
from sklearn.decomposition import PCA

In [177]:
PCA_N_COMPONENTS = 0.5
PCANAME = 'pca_200_50.pkl'
CLF1NAME = 'person_pre_eliminary_200_50.pkl'
CLF2NAME = 'person_final_200_50.pkl'

In [178]:
import sys
import os
import cv2
import numpy as np
from sklearn.svm import LinearSVC
import joblib
from skimage.feature import hog
from sklearn.utils import shuffle
import argparse
import random
from sklearn.decomposition import PCA
MAX_HARD_NEGATIVES = 200

# parser = argparse.ArgumentParser(description='Parse Training Directory')
# parser.add_argument('--pos', help='Path to directory containing Positive Images')
# parser.add_argument('--neg', help='Path to directory containing Negative images')
#python3 train.py --pos INRIAPerson/train_64x128_H96/pos --neg INRIAPerson/train_64x128_H96/neg
# args = parser.parse_args()
pos_img_dir = 'INRIAPerson/train_64x128_H96/pos'
neg_img_dir = 'INRIAPerson/train_64x128_H96/neg'

def crop_centre(img):
    h, w, _ = img.shape
    l = (w - 64) // 2
    t = (h - 128) // 2

    crop = img[t:t+128, l:l+64]
    return crop

def ten_random_windows(img):
    h, w = img.shape
    if h < 128 or w < 64:
        return []

    h = h - 128
    w = w - 64

    windows = []

    for i in range(10):
        x = random.randint(0, w)
        y = random.randint(0, h)
        windows.append(img[y:y+128, x:x+64])

    return windows

def read_filenames():

    f_pos = []
    f_neg = []

    mypath_pos = pos_img_dir
    for (dirpath, dirnames, filenames) in os.walk(mypath_pos):
        f_pos.extend(filenames)
        break

    mypath_neg = neg_img_dir
    for (dirpath, dirnames, filenames) in os.walk(mypath_neg):
        f_neg.extend(filenames)
        break

    return f_pos, f_neg

def read_images(pos_files, neg_files):

    X = []
    Y = []

    pos_count = 0

    for img_file in pos_files:
        print(os.path.join(pos_img_dir, img_file))
        img = cv2.imread(os.path.join(pos_img_dir, img_file))

        cropped = crop_centre(img)

        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        features = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2", transform_sqrt=True, feature_vector=True)
        pos_count += 1

        X.append(features)
        Y.append(1)

    neg_count = 0

    for img_file in neg_files:
        print(os.path.join(neg_img_dir, img_file))
        img = cv2.imread(os.path.join(neg_img_dir, img_file))
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        windows = ten_random_windows(gray_img)

        for win in windows:
            features = hog(win, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2", transform_sqrt=True, feature_vector=True)
            neg_count += 1
            X.append(features)
            Y.append(0)

    return X, Y, pos_count, neg_count

def sliding_window(image, window_size, step_size):
    for y in range(0, image.shape[0] - window_size[1], step_size[1]):
        for x in range(0, image.shape[1] - window_size[0], step_size[0]):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

def hard_negative_mine(f_neg, winSize, winStride):

    hard_negatives = []
    hard_negative_labels = []

    count = 0
    num = 0
    for imgfile in f_neg:
        img = cv2.imread(os.path.join(neg_img_dir, imgfile))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        for (x, y, im_window) in sliding_window(gray, winSize, winStride):
            features = hog(im_window, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2", transform_sqrt=True, feature_vector=True)
            # print('asf')
            features_pca = pca.transform(features.reshape(1, -1))
            if (clf1.predict(features_pca) == 1):
                hard_negatives.append(features_pca.flatten())
                hard_negative_labels.append(0)
                count = count + 1

            if (count == MAX_HARD_NEGATIVES):
                return np.array(hard_negatives), np.array(hard_negative_labels)

        num = num + 1

        sys.stdout.write("\r" + "\tHard Negatives Mined: " + str(count) + "\tCompleted: " + str(round((count / float(MAX_HARD_NEGATIVES))*100, 4)) + " %")

        sys.stdout.flush()

    return np.array(hard_negatives), np.array(hard_negative_labels)



In [179]:
pos_img_files, neg_img_files = read_filenames()

print( "Total Positive Images : " + str(len(pos_img_files)))
print("Total Negative Images : " + str(len(neg_img_files)))
print("Reading Images")

X, Y, pos_count, neg_count = read_images(pos_img_files, neg_img_files)

X = np.array(X)
Y = np.array(Y)

# X, Y = shuffle(X, Y, random_state=0)


Total Positive Images : 2416
Total Negative Images : 1218
Reading Images
INRIAPerson/train_64x128_H96/pos\crop001001a.png
INRIAPerson/train_64x128_H96/pos\crop001001b.png
INRIAPerson/train_64x128_H96/pos\crop001001c.png
INRIAPerson/train_64x128_H96/pos\crop001001d.png
INRIAPerson/train_64x128_H96/pos\crop001001e.png
INRIAPerson/train_64x128_H96/pos\crop001001f.png
INRIAPerson/train_64x128_H96/pos\crop001002a.png
INRIAPerson/train_64x128_H96/pos\crop001002b.png
INRIAPerson/train_64x128_H96/pos\crop001002c.png
INRIAPerson/train_64x128_H96/pos\crop001002d.png
INRIAPerson/train_64x128_H96/pos\crop001003a.png
INRIAPerson/train_64x128_H96/pos\crop001003b.png
INRIAPerson/train_64x128_H96/pos\crop001003c.png
INRIAPerson/train_64x128_H96/pos\crop001003d.png
INRIAPerson/train_64x128_H96/pos\crop001004a.png
INRIAPerson/train_64x128_H96/pos\crop001004b.png
INRIAPerson/train_64x128_H96/pos\crop001004c.png
INRIAPerson/train_64x128_H96/pos\crop001004d.png
INRIAPerson/train_64x128_H96/pos\crop001005a.

In [180]:
pca = PCA(n_components=PCA_N_COMPONENTS) 
pca.fit(X)
X_pca = pca.transform(X)

In [181]:
joblib.dump(pca, PCANAME)

['pca_200_50.pkl']

In [182]:

print("Images Read and Shuffled")
print("Positives: " + str(pos_count))
print("Negatives: " + str(neg_count))
print("Training Started")

clf1 = LinearSVC(C=0.01, max_iter=1000, class_weight='balanced', verbose=1)

clf1.fit(X_pca, Y)
print("Trained")

joblib.dump(clf1, CLF1NAME)

print("Hard Negative Mining")

winStride = (8, 8)
winSize = (64, 128)

print("Maximum Hard Negatives to Mine: " + str(MAX_HARD_NEGATIVES))

hard_negatives, hard_negative_labels = hard_negative_mine(neg_img_files, winSize, winStride)


Images Read and Shuffled
Positives: 2416
Negatives: 12180
Training Started
[LibLinear]Trained
Hard Negative Mining
Maximum Hard Negatives to Mine: 200


C:\Users\lk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


	Hard Negatives Mined: 197	Completed: 98.5 %

In [183]:
hard_negatives1 = hard_negatives
hard_negatives1 = np.array([hn.flatten() for hn in hard_negatives1])
hard_negatives1 = np.concatenate((hard_negatives1, X_pca), axis=0)
hard_negative_labels = np.concatenate((hard_negative_labels, Y), axis=0)

hard_negatives1, hard_negative_labels = shuffle(hard_negatives1, hard_negative_labels, random_state=0)

In [184]:


clf2 = LinearSVC(C=0.01, max_iter=1000, class_weight='balanced', verbose=1)

clf2.fit(hard_negatives1, hard_negative_labels)

print("Trained and Dumping")

joblib.dump(clf2, CLF2NAME)

[LibLinear]Trained and Dumping


C:\Users\lk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


['person_final_200_50.pkl']